In [1]:
%load_ext autoreload

# This is just in case logging configuration needs each cell to "refresh".

# NOTE: I don't think this was actually necessary.

Following [README](https://github.com/Delgan/loguru#readme)

## Summary
`Loguru` is a library which aims to bring enjoyable logging in Python.

Did you ever feel lazy about configuring a logger and used `print()` instead?...

I did, yet logging is fundamental to every application and eases the process of debugging.

Using `Loguru` you have no excuse not to use logging from the start, this is as simple as `from loguru import logger`.

## Take the tour

### Ready to use out of the box without boilerplate

> The main concept of Loguru is that there is one and only one `logger`.

For convenience, it is pre-configured and outputs to `stderr` to begin with (but that's entirely configurable).

The [`logger`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger)
is just an interface which dispatches log messages to configured handlers. Simple, right?

In [2]:
from loguru import logger

logger.debug("That's it, beautiful and simple logging!")

2022-12-29 22:17:06.443 | DEBUG    | __main__:<module>:3 - That's it, beautiful and simple logging!


### No `Handler`, no `Formatter`, no `Filter`: *one function to rule them all*

How to add a handler? How to set up logs formatting? How to filter messages? How to set level?

One answer: the [`add()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.add) function.

In [3]:
%autoreload

import sys

l2 = logger.add(
    sys.stderr, format=">>> {time} {level} {message}", filter="__main__", level="INFO"
)

logger.info("That's it, beautiful and simple logging!")

2022-12-29 22:17:06.522 | INFO     | __main__:<module>:7 - That's it, beautiful and simple logging!
>>> 2022-12-29T22:17:06.522543+0000 INFO That's it, beautiful and simple logging!


This function should be used to register [**sinks**](https://loguru.readthedocs.io/en/stable/api/logger.html#sink) which
are responsible for managing [**log messages**](https://loguru.readthedocs.io/en/stable/api/logger.html#message)
contextualized with a [**record dict**](https://loguru.readthedocs.io/en/stable/api/logger.html#record).

A sink can take many forms:
* a simple function,
* a string path,
* a file-like object,
* a coroutine function
* or a built-in Handler.

Note that you may also [`remove()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.remove)
a previously added handler by using the identifier returned while adding it.

This is particularly useful if you *want to supersede the default stderr handler*:
* just call `logger.remove()` to make a fresh start.

In [4]:
logger.remove(l2)

### Easier file logging with rotation / retention / compression

If you want to send logged messages to a file, you just have to use a string path as the sink.

It can be automatically timed too for convenience:

In [5]:
%autoreload

from loguru import logger

l3 = logger.add("file_{time}.log")  # NOTE the automatically handled {time} macro here.

logger.debug("That's it, beautiful and simple logging!")

logger.remove(l3)

2022-12-29 22:17:06.692 | DEBUG    | __main__:<module>:7 - That's it, beautiful and simple logging!


It is also [easily configurable](https://loguru.readthedocs.io/en/stable/api/logger.html#file) if you need rotating logger,
if you want to remove older logs, or if you wish to compress your files at closure.

```python
logger.add("file_1.log", rotation="500 MB")    # Automatically rotate too big file
logger.add("file_2.log", rotation="12:00")     # New file is created each day at noon
logger.add("file_3.log", rotation="1 week")    # Once the file is too old, it's rotated

logger.add("file_X.log", retention="10 days")  # Cleanup after some time

logger.add("file_Y.log", compression="zip")    # Save some loved space
```

### Modern string formatting using braces style

Loguru favors the much more elegant and powerful `{}` formatting over `%`,
**logging functions are actually equivalent to `str.format()`**.

* If using python 3.6+, use f-strings.

In [6]:
logger.info(
    "If you're using Python {}, prefer {feature} of course!", 3.6, feature="f-strings"
)

2022-12-29 22:17:06.769 | INFO     | __main__:<module>:1 - If you're using Python 3.6, prefer f-strings of course!


### Exceptions catching within threads or main

⚠️ Tricky

* Have you ever seen your program crashing unexpectedly without seeing anything in the log file?
* Did you ever notice that exceptions occurring in threads were not logged?

This can be solved using the [`catch()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.catch)
decorator / context manager which ensures that any error is correctly propagated to the logger.

In [7]:
@logger.catch
def my_function(x, y, z):
    # An error? It's caught anyway!
    return 1 / (x + y + z)


my_function(0, 0, 0)

2022-12-29 22:17:06.850 | ERROR    | __main__:<module>:6 - An error has been caught in function '<module>', process 'MainProcess' (1505681), thread 'MainThread' (140272710476864):
Traceback (most recent call last):

  File "/mnt/data-linux/miniconda3/envs/learn_py38_loguru/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7f93c8a4d9d0, file "/mnt/data-linux/miniconda3/envs/learn_py38_loguru/lib/python3.8/site-packages/...
           └ <function _run_code at 0x7f93c8a51280>
  File "/mnt/data-linux/miniconda3/envs/learn_py38_loguru/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from th

### Pretty logging with colors

Loguru automatically adds colors to your logs if your terminal is compatible.

You can define your favorite style by using markup tags in the sink format.

* Markup tags: https://loguru.readthedocs.io/en/stable/api/logger.html#color

In [8]:
%autoreload

from loguru import logger

l4 = logger.add(
    sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>"
)

logger.debug("That's it, beautiful and simple logging!")

logger.remove(l4)

2022-12-29 22:17:06.932 | DEBUG    | __main__:<module>:7 - That's it, beautiful and simple logging!


2022-12-29T22:17:06.932908+0000 That's it, beautiful and simple logging!


### Asynchronous, Thread-safe, Multiprocess-safe

All sinks added to the logger are thread-safe by default.

They are not multiprocess-safe, but you can `enqueue` the messages to ensure logs integrity.

This same argument can also be used if you want async logging.

```python
# Like so:
logger.add("somefile.log", enqueue=True)
```

Coroutine functions used as sinks are also supported and should be awaited with
[`complete()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.complete).

### Fully descriptive exceptions

Logging exceptions that occur in your code is important to track bugs, but it's quite useless if you don't know why it failed.

Loguru helps you identify problems by *allowing the entire stack trace to be displayed*, including **values of variables**
(thanks [`better_exceptions`](https://github.com/Qix-/better-exceptions) for this!).

> Note that this feature won't work on default Python REPL due to unavailable frame data.

The code:

In [9]:
%autoreload

from loguru import logger

l5 = logger.add(
    "out.log", backtrace=True, diagnose=True
)  # Caution, may leak sensitive data in prod


def func(a, b):
    return a / b


def nested(c):
    try:
        func(5, c)
    except ZeroDivisionError:
        logger.exception("What?!")


nested(0)

logger.remove(l5)

# NOTE: See also out.log

2022-12-29 22:17:07.010 | ERROR    | __main__:nested:14 - What?!
Traceback (most recent call last):

  File "/mnt/data-linux/miniconda3/envs/learn_py38_loguru/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7f93c8a4d9d0, file "/mnt/data-linux/miniconda3/envs/learn_py38_loguru/lib/python3.8/site-packages/...
           └ <function _run_code at 0x7f93c8a51280>
  File "/mnt/data-linux/miniconda3/envs/learn_py38_loguru/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code object <module> at 0x7f93c8a4d9d0, file "/mnt/data-linux/miniconda3/envs/learn

### Structured logging as needed

Want your logs to be serialized for easier parsing or to pass them around?

Using the `serialize` argument, each log message *will be converted to a JSON string* before being sent to the configured sink.


In [12]:
def custom_sink_function(x):
    display(type(x))
    print(x)


l6 = logger.add(custom_sink_function, serialize=True)

logger.info("my message")

logger.remove(l6)

2022-12-29 22:17:22.185 | INFO     | __main__:<module>:7 - my message


loguru._handler.Message

{"text": "2022-12-29 22:17:22.185 | INFO     | __main__:<module>:7 - my message\n", "record": {"elapsed": {"repr": "0:00:15.743776", "seconds": 15.743776}, "exception": null, "extra": {}, "file": {"name": "3367625739.py", "path": "/tmp/ipykernel_1505681/3367625739.py"}, "function": "<module>", "level": {"icon": "ℹ️", "name": "INFO", "no": 20}, "line": 7, "message": "my message", "module": "3367625739", "name": "__main__", "process": {"id": 1505681, "name": "MainProcess"}, "thread": {"id": 140272710476864, "name": "MainThread"}, "time": {"repr": "2022-12-29 22:17:22.185390+00:00", "timestamp": 1672352242.18539}}}



Using [`bind()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.bind) you can
contextualize your logger messages *by modifying the extra record attribute*.

In [13]:
l7 = logger.add("file.log", format="{extra[ip]} {extra[user]} {message}")

context_logger = logger.bind(ip="192.168.0.1", user="someone")
context_logger.info("Contextualize your logger easily")
context_logger.bind(user="someone_else").info("Inline binding of extra attribute")
context_logger.info(
    "Use kwargs to add context during formatting: {user}", user="anybody"
)

# NOTE: See file.log for output with contextualization.

logger.remove(l7)

2022-12-29 22:18:43.005 | INFO     | __main__:<module>:4 - Contextualize your logger easily
2022-12-29 22:18:43.005 | INFO     | __main__:<module>:5 - Inline binding of extra attribute
2022-12-29 22:18:43.006 | INFO     | __main__:<module>:6 - Use kwargs to add context during formatting: anybody


It is possible to modify a context-local state temporarily with `contextualize()`:
```python
with logger.contextualize(task=task_id):
    do_something()
    logger.info("End of task")
```

You can also have more fine-grained control over your logs by combining `bind()` and `filter`:
```python
logger.add("special.log", filter=lambda record: "special" in record["extra"])
logger.debug("This message is not logged to the file")
logger.bind(special=True).info("This message, though, is logged to the file!")
```

Finally, the [`patch()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.patch)
method allows dynamic values to be attached to the record dict of each new message:
```python
logger.add(sys.stderr, format="{extra[utc]} {message}")
logger = logger.patch(lambda record: record["extra"].update(utc=datetime.utcnow()))
```

### Lazy evaluation of expensive functions and `opt()`

Sometime you would like to log verbose information without performance penalty in production, you can use the `opt()` method to achieve this.

```python
logger.opt(lazy=True).debug("If sink level <= DEBUG: {x}", x=lambda: expensive_function(2**64))

# By the way, "opt()" serves many usages
logger.opt(exception=True).info("Error stacktrace added to the log message (tuple accepted too)")
logger.opt(colors=True).info("Per message <blue>colors</blue>")
logger.opt(record=True).info("Display values from the record (eg. {record[thread]})")
logger.opt(raw=True).info("Bypass sink formatting\n")
logger.opt(depth=1).info("Use parent stack context (useful within wrapped functions)")
logger.opt(capture=False).info("Keyword arguments not added to {dest} dict", dest="extra")
```

### Customizable levels

Loguru comes with all standard [logging levels](https://loguru.readthedocs.io/en/stable/api/logger.html#levels)
to which 
* [`trace()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.trace) and 
* [`success()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.success)

are added.

Do you need more? Then, just create it by using the [`level()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.level) function.

### Better datetime handling

The standard logging is bloated with arguments like `datefmt` or `msecs`, `%(asctime)s` and `%(created)s`,
naive datetimes without timezone information, not intuitive formatting, etc.

Loguru fixes it:

In [15]:
l8 = logger.add(
    "file2.log", format="{time:YYYY-MM-DD at HH:mm:ss} | {level} | {message}"
)  # pyright: ignore

context_logger.info("Something")

# NOTE: See file2.log for output with contextualization.

logger.remove(l8)

2022-12-29 22:27:12.539 | INFO     | __main__:<module>:3 - Something


### Suitable for scripts and libraries

Using the logger in your scripts is easy, and you can [`configure()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.configure) it at start.

To use Loguru from **inside a library**, remember to never call [`add()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.add) 
but use [`disable()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.disable) instead so logging
functions become no-op.

If a developer wishes to see your library's logs, he can [`enable()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.enable) it again.

> [❗] So in Loguru's opinion, you should not have your library showing logs by default!

Illustration:
```python
# For scripts
config = {
    "handlers": [
        {"sink": sys.stdout, "format": "{time} - {message}"},
        {"sink": "file.log", "serialize": True},
    ],
    "extra": {"user": "someone"}
}
logger.configure(**config)

# For libraries
logger.disable("my_library")
logger.info("No matter added sinks, this message is not displayed")
logger.enable("my_library")
logger.info("This message however is propagated to the sinks")
```

### Entirely compatible with standard logging

Wish to use built-in logging `Handler` as a Loguru sink?

```python
# Like so:
handler = logging.handlers.SysLogHandler(address=('localhost', 514))
logger.add(handler)
```

> ⚠️ Very advanced - probably needs understanding of the advanced aspects of the Python logging ecosystem. 

Need to propagate Loguru messages to standard logging?
```python
# Like so:
class PropagateHandler(logging.Handler):
    def emit(self, record):
        logging.getLogger(record.name).handle(record)

logger.add(PropagateHandler(), format="{message}")
```

Want to intercept standard logging messages toward your Loguru sinks?
```python
# Like so:

class InterceptHandler(logging.Handler):
    def emit(self, record):
        # Get corresponding Loguru level if it exists.
        try:
            level = logger.level(record.levelname).name
        except ValueError:
            level = record.levelno

        # Find caller from where originated the logged message.
        frame, depth = sys._getframe(6), 6
        while frame and frame.f_code.co_filename == logging.__file__:
            frame = frame.f_back
            depth += 1

        logger.opt(depth=depth, exception=record.exc_info).log(level, record.getMessage())

logging.basicConfig(handlers=[InterceptHandler()], level=0, force=True)
```

### Personalizable defaults through environment variables

Don't like the default logger formatting? Would prefer another `DEBUG` color?

No problem:
```sh
# Linux / OSX
export LOGURU_FORMAT="{time} | <lvl>{message}</lvl>"

# Windows
setx LOGURU_DEBUG_COLOR "<green>"
```

* Info on env var configuration here: https://loguru.readthedocs.io/en/stable/api/logger.html#env

### Convenient parser

It is often useful to extract specific information from generated logs, this is why Loguru provides a
[`parse()`](https://loguru.readthedocs.io/en/stable/api/logger.html#loguru._logger.Logger.parse) method which helps
to deal with logs and regexes.

```python
# Example.

pattern = r"(?P<time>.*) - (?P<level>[0-9]+) - (?P<message>.*)"  # Regex with named groups
caster_dict = dict(time=dateutil.parser.parse, level=int)        # Transform matching groups

for groups in logger.parse("file.log", pattern, cast=caster_dict):
    print("Parsed:", groups)
    # {"level": 30, "message": "Log example", "time": datetime(2018, 12, 09, 11, 23, 55)}
```

### Exhaustive notifier

Loguru can easily be combined with the great [notifiers](https://github.com/notifiers/notifiers) library.